## IMPORT

In [1]:
pip install --upgrade pip


The following command must be run outside of the IPython shell:

    $ pip install --upgrade pip

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
!pip install pmdarima

In [3]:
!pip install seaborn

In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

In [50]:
train_dangjin_floating = pd.read_csv('dangjin_floating_train_0526.csv')
train_dangjin_warehouse = pd.read_csv('dangjin_warehouse_train_0526.csv')
train_dangjin = pd.read_csv('dangjin_train_0526.csv')
train_ulsan = pd.read_csv('ulsan_train_0526.csv')

test_dangjin_floating = pd.read_csv('dangjin_floating_test_0526.csv')
test_dangjin_warehouse = pd.read_csv('dangjin_warehouse_test_0526.csv')
test_dangjin = pd.read_csv('dangjin_test_0526.csv')
test_ulsan = pd.read_csv('ulsan_test_0526.csv')

train_ulsan = pd.read_csv('ulsan_train_0526.csv',index_col='date',parse_dates=True)

## energy na값 잘라내기 (train dangjin_floating, dangjin_warehouse)

dangjin_warehouse는 2018년 3월 17일, 18일이 다 비어있음.

index 336부터 383까지

In [5]:
train_dangjin_warehouse=train_dangjin_warehouse.drop(list(range(336,384,1)))

In [6]:
train_dangjin_warehouse[train_dangjin_warehouse['energy'].isna()]

,date,T3,Wx3,Wy3,H3,C3,dp3,FD3,T6,Wx6,...,dayofyear,hour,sin_hour,cos_hour,sin_dayofyear,cos_dayofyear,sin(elev ang),sin(azimuth ang),cos(azimuth ang),energy


dangjin_floating는 2020년 6월 26일이 다 비어있음.

index 20280부터 20303까지

In [7]:
train_dangjin_floating=train_dangjin_floating.drop(list(range(20280,20304,1)))

In [8]:
train_dangjin_floating[train_dangjin_floating['energy'].isna()]

,date,T3,Wx3,Wy3,H3,C3,dp3,FD3,T6,Wx6,...,dayofyear,hour,sin_hour,cos_hour,sin_dayofyear,cos_dayofyear,sin(elev ang),sin(azimuth ang),cos(azimuth ang),energy


## PCA

### ->이제 principal DF로 독립변수 새롭게 밑에서 arima돌릴 때 exogenous 변수로 사용.

In [8]:
from sklearn.decomposition import PCA

In [9]:
pd.set_option('display.max_columns',None)

In [10]:
corrMatrix = train_ulsan.corr()

### ulsan

#### train

In [51]:
train_ulsan.columns

Index(['date', 'T3', 'Wx3', 'Wy3', 'H3', 'C3', 'dp3', 'FD3', 'T6', 'Wx6',
       'Wy6', 'H3.1', 'C6', 'dp6', 'FD6', 'T12', 'Wx12', 'Wy12', 'H12', 'C12',
       'dp12', 'FD12', 'T24', 'Wx24', 'Wy24', 'H24', 'C24', 'dp24', 'FD24',
       'Temperature', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y', 'dew_point(째C)',
       'dayofyear', 'hour', 'sin_hour', 'cos_hour', 'sin_dayofyear',
       'cos_dayofyear', 'sin(elev ang)', 'sin(azimuth ang)',
       'cos(azimuth ang)', 'energy'],
      dtype='object')

In [52]:
train_ulsan_x=train_ulsan[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [53]:
train_ulsan_y=train_ulsan['energy']

In [54]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(train_ulsan_x)

In [55]:
pca.explained_variance_ratio_

array([0.6218552 , 0.26754622])

In [56]:
sum(pca.explained_variance_ratio_)

0.8894014134647164

In [57]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [58]:
train_ulsan['principal component1']=principalDf['principal component1']

In [59]:
train_ulsan['principal component2']=principalDf['principal component2']

#### test

In [60]:
test_ulsan_x=test_ulsan[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [61]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(test_ulsan_x)

In [62]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [63]:
sum(pca.explained_variance_ratio_)

0.9348094785303671

In [64]:
test_ulsan['principal component1']=principalDf['principal component1']

In [65]:
test_ulsan['principal component2']=principalDf['principal component2']

### dangjin

#### train

In [26]:
train_dangjin_x=train_dangjin[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [27]:
train_dangjin_y=train_dangjin['energy']

In [28]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(train_dangjin_x)

In [29]:
pca.explained_variance_ratio_

array([0.54160469, 0.31689256])

In [30]:
sum(pca.explained_variance_ratio_)

0.8584972521962222

In [31]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [32]:
train_dangjin['principal component1']=principalDf['principal component1']

In [33]:
train_dangjin['principal component2']=principalDf['principal component2']

#### test

In [34]:
test_dangjin_x=test_dangjin[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [35]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(test_dangjin_x)

In [36]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [37]:
test_dangjin['principal component1']=principalDf['principal component1']

In [38]:
test_dangjin['principal component2']=principalDf['principal component2']

### dangjin_floating

#### train

In [77]:
train_dangjin_floating=train_dangjin_floating.reset_index()

In [78]:
train_dangjin_floating_x=train_dangjin_floating[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [79]:
train_dangjin_floating_y=train_dangjin_floating['energy']

In [80]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(train_dangjin_floating_x)

In [81]:
pca.explained_variance_ratio_

array([0.54160469, 0.31689256])

In [82]:
sum(pca.explained_variance_ratio_)

0.8584972521962219

In [83]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [84]:
train_dangjin_floating['principal component1']=principalDf['principal component1']

In [85]:
train_dangjin_floating['principal component2']=principalDf['principal component2']

#### test

In [86]:
test_dangjin_floating_x=test_dangjin_floating[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [87]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(test_dangjin_floating_x)

In [88]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [89]:
test_dangjin_floating['principal component1']=principalDf['principal component1']

In [90]:
test_dangjin_floating['principal component2']=principalDf['principal component2']

### dangjin_warehouse

#### train

In [54]:
train_dangjin_warehouse=train_dangjin_warehouse.reset_index()

In [55]:
train_dangjin_warehouse_x=train_dangjin_warehouse[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [56]:
train_dangjin_warehouse_y=train_dangjin_warehouse['energy']

In [57]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(train_dangjin_warehouse_x)

In [58]:
pca.explained_variance_ratio_

array([0.54186321, 0.31672589])

In [59]:
sum(pca.explained_variance_ratio_)

0.8585890965562766

In [60]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [61]:
train_dangjin_warehouse['principal component1']=principalDf['principal component1']

In [62]:
train_dangjin_warehouse['principal component2']=principalDf['principal component2']

#### test

In [63]:
test_dangjin_warehouse_x=test_dangjin_warehouse[['T3','Wx3','Wy3','H3','C3','dp3','FD3','cos_hour',
                           'sin(azimuth ang)','sin_dayofyear']]

In [64]:
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
principalComponents = pca.fit_transform(test_dangjin_warehouse_x)

In [65]:
principalDf = pd.DataFrame(data=principalComponents, columns = ['principal component1', 'principal component2'])

In [66]:
test_dangjin_warehouse['principal component1']=principalDf['principal component1']

In [67]:
test_dangjin_warehouse['principal component2']=principalDf['principal component2']

## Sarimax

### index에 freq 설정

In [39]:
train_ulsan.index=pd.to_datetime(train_ulsan['date'])
test_ulsan.index=pd.to_datetime(test_ulsan['date'])

In [40]:
train_ulsan.index.freq='H'
test_ulsan.index.freq='H'

#### DATE index 수정

#### dangjin

In [88]:
index_goal=train_ulsan['date']

In [67]:
index_goal=pd.DataFrame(index_goal)

In [79]:
dangjin_index_goal = pd.merge(index_goal, train_dangjin,how='left',on='date')

In [82]:
dangjin_index_goal.index=pd.to_datetime(dangjin_index_goal['date'])
test_dangjin.index=pd.to_datetime(test_dangjin['date'])

In [83]:
dangjin_index_goal.index.freq='H'
test_dangjin.index.freq='H'

In [85]:
dangjin_index_goal1=dangjin_index_goal.drop(['date'],axis=1)

In [86]:
dangjin_index_goal2=dangjin_index_goal1.dropna(how='all')

In [ ]:
train_dangjin.index.freq='H'

In [205]:
test_dangjin.index.freq='H'

In [94]:
train_dangjin_floating.index=pd.to_datetime(train_dangjin_floating['date'])
test_dangjin_floating.index=pd.to_datetime(test_dangjin_floating['date'])

In [206]:
test_dangjin_floating.index.freq='H'

In [278]:
train_dangjin_warehouse.index=pd.to_datetime(train_dangjin_warehouse['date'])
test_dangjin_warehouse.index=pd.to_datetime(test_dangjin_warehouse['date'])

In [ ]:
train_dangjin_warehouse.index.freq='H'

In [207]:
test_dangjin_warehouse.index.freq='H'

### ulsan

In [39]:
%matplotlib inline

In [114]:
import matplotlib.dates as dates

result=seasonal_decompose(train_ulsan['energy'])
result.plot()

In [ ]:
auto_arima(train_ulsan['energy'],exogenous=train_ulsan[['principal component1','principal component2']],seasonal=True, m=365).summary()

In [234]:
model=SARIMAX(train_ulsan['energy'],exog=train_ulsan[['principal component1','principal component2']],order=(5,1,0),
                       seasonal_order=(2, 0, 0, 365),enforce_invertibiblity=False)

results=model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                             energy   No. Observations:                25584
Model:             SARIMAX(5, 1, 0)x(2, 0, 0, 12)   Log Likelihood             -122535.379
Date:                            Wed, 02 Jun 2021   AIC                         245090.758
Time:                                    11:26:02   BIC                         245172.255
Sample:                                03-03-2018   HQIC                        245117.107
                                     - 01-31-2021                                         
Covariance Type:                              opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.5323      0.063      8.393      0.000       0.408       0.657
principal component2    -0.1037      0.041     -2.524      0.012      -0.184      -0.023
ar.L1                    0.2930      0.004     83.346      0.000       0.286       0.300
ar.L2                    0.0586      0.004     15.131      0.000       0.051       0.066
ar.L3                   -0.0304      0.004     -7.301      0.000      -0.039      -0.022
ar.L4                   -0.0491      0.005    -10.274      0.000      -0.059      -0.040
ar.L5                   -0.1455      0.005    -29.221      0.000      -0.155      -0.136
ar.S.L12                -0.0644      0.019     -3.358      0.001      -0.102      -0.027
ar.S.L24                 0.4401      0.004    121.574      0.000       0.433       0.447
sigma2                 846.7431      3.726    227.242      0.000     839.440     854.046
===================================================================================
Ljung-Box (L1) (Q):                   9.10   Jarque-Bera (JB):             66457.25
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.11   Skew:                             0.06
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.90
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [235]:
fcast1=results.predict(0,len(test_ulsan),exog=test_ulsan[['principal component1','principal component2']]).rename("FINAL SARIMAX FORECAST")

### dangjin

In [192]:
auto_arima(train_dangjin['energy'],exogenous=train_dangjin[['principal component1','principal component2']],seasonal=True, m=12).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                25560
Model:             SARIMAX(1, 1, 0)x(2, 0, 0, 12)   Log Likelihood             -142989.281
Date:                            Wed, 02 Jun 2021   AIC                         285990.562
Time:                                    02:54:20   BIC                         286039.455
Sample:                                         0   HQIC                        286006.370
                                          - 25560                                         
Covariance Type:                              opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.2270      0.075      3.018      0.003       0.080       0.374
principal component2    -1.2957      0.114    -11.362      0.000      -1.519      -1.072
ar.L1                    0.3411      0.003    100.344      0.000       0.334       0.348
ar.S.L12                 0.0107      0.045      0.236      0.813      -0.078       0.100
ar.S.L24                 0.4658      0.003    138.924      0.000       0.459       0.472
sigma2                4233.6383     17.818    237.600      0.000    4198.715    4268.562
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             66447.27
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):               0.99   Skew:                            -0.37
Prob(H) (two-sided):                  0.50   Kurtosis:                        10.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [193]:
model=SARIMAX(train_dangjin['energy'],exog=train_dangjin[['principal component1','principal component2']],order=(1,1,0),
                       seasonal_order=(2, 0, 0, 12),enforce_invertibiblity=False)

results=model.fit()

results.summary()

/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                             energy   No. Observations:                25560
Model:             SARIMAX(1, 1, 0)x(2, 0, 0, 12)   Log Likelihood             -142989.281
Date:                            Wed, 02 Jun 2021   AIC                         285990.562
Time:                                    03:03:16   BIC                         286039.455
Sample:                                         0   HQIC                        286006.370
                                          - 25560                                         
Covariance Type:                              opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.2270      0.075      3.018      0.003       0.080       0.374
principal component2    -1.2957      0.114    -11.362      0.000      -1.519      -1.072
ar.L1                    0.3411      0.003    100.344      0.000       0.334       0.348
ar.S.L12                 0.0107      0.045      0.236      0.813      -0.078       0.100
ar.S.L24                 0.4658      0.003    138.924      0.000       0.459       0.472
sigma2                4233.6383     17.818    237.600      0.000    4198.715    4268.562
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             66447.27
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):               0.99   Skew:                            -0.37
Prob(H) (two-sided):                  0.50   Kurtosis:                        10.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [194]:
fcast2=results.predict(0,len(test_dangjin),exog=test_dangjin[['principal component1','principal component2']]).rename("FINAL SARIMAX FORECAST")

### dangjin_warehouse


In [195]:
auto_arima(train_dangjin_warehouse['energy'],exogenous=train_dangjin_warehouse[['principal component1','principal component2']],seasonal=True, m=12).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                25512
Model:             SARIMAX(0, 1, 4)x(2, 0, [], 12)   Log Likelihood             -132059.575
Date:                             Wed, 02 Jun 2021   AIC                         264137.149
Time:                                     05:31:53   BIC                         264210.471
Sample:                                          0   HQIC                        264160.858
                                           - 25512                                         
Covariance Type:                               opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.1628      0.050      3.235      0.001       0.064       0.261
principal component2    -0.9237      0.077    -11.946      0.000      -1.075      -0.772
ma.L1                    0.3464      0.003    101.876      0.000       0.340       0.353
ma.L2                    0.1466      0.004     34.906      0.000       0.138       0.155
ma.L3                    0.0303      0.005      6.250      0.000       0.021       0.040
ma.L4                   -0.0188      0.005     -3.716      0.000      -0.029      -0.009
ar.S.L12                 0.0044      0.047      0.093      0.926      -0.088       0.097
ar.S.L24                 0.4460      0.003    131.766      0.000       0.439       0.453
sigma2                1835.4179      7.654    239.803      0.000    1820.417    1850.419
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             73791.10
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               1.22   Skew:                            -0.38
Prob(H) (two-sided):                  0.00   Kurtosis:                        11.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [279]:
model=SARIMAX(train_dangjin_warehouse['energy'],exog=train_dangjin_warehouse[['principal component1','principal component2']],order=(0,1,4),
                       seasonal_order=(2, 0, 0, 12),enforce_invertibiblity=False)

results=model.fit()

results.summary()

/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                              energy   No. Observations:                25512
Model:             SARIMAX(0, 1, 4)x(2, 0, [], 12)   Log Likelihood             -132059.575
Date:                             Wed, 02 Jun 2021   AIC                         264137.149
Time:                                     14:35:48   BIC                         264210.471
Sample:                                          0   HQIC                        264160.858
                                           - 25512                                         
Covariance Type:                               opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.1628      0.050      3.235      0.001       0.064       0.261
principal component2    -0.9237      0.077    -11.946      0.000      -1.075      -0.772
ma.L1                    0.3464      0.003    101.876      0.000       0.340       0.353
ma.L2                    0.1466      0.004     34.906      0.000       0.138       0.155
ma.L3                    0.0303      0.005      6.250      0.000       0.021       0.040
ma.L4                   -0.0188      0.005     -3.716      0.000      -0.029      -0.009
ar.S.L12                 0.0044      0.047      0.093      0.926      -0.088       0.097
ar.S.L24                 0.4460      0.003    131.766      0.000       0.439       0.453
sigma2                1835.4179      7.654    239.803      0.000    1820.417    1850.419
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             73791.10
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               1.22   Skew:                            -0.38
Prob(H) (two-sided):                  0.00   Kurtosis:                        11.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [280]:
fcast3=results.predict(0,len(test_dangjin_warehouse),exog=test_dangjin_warehouse[['principal component1','principal component2']]).rename("FINAL SARIMAX FORECAST")

### dangjin_floating

In [96]:
train_dangjin_floating

,index,date,T3,Wx3,Wy3,H3,C3,dp3,FD3,T6,Wx6,Wy6,H3.1,C6,dp6,FD6,T12,Wx12,Wy12,H12,C12,dp12,FD12,T24,Wx24,Wy24,H24,C24,dp24,FD24,Temperature,Humidity,Cloud,Wind_x,Wind_y,dew_point(째C),dayofyear,hour,sin_hour,cos_hour,sin_dayofyear,cos_dayofyear,sin(elev ang),sin(azimuth ang),cos(azimuth ang),energy,principal component1,principal component2
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-03-03 00:00:00,0,2018-03-03 0:00,-2.300000,3.234936,-1.128891e+00,57.000000,0.600000,-9.633333,42.000000,-2.033333,3.328301,-2.000005,59.666667,0.583333,-8.816667,49.666667,-0.975000,3.416215,-2.900476,58.916667,0.541667,-7.975000,57.416667,0.450000,3.279195,-3.302690,70.958333,0.554167,-4.454167,49.208333,-2.000000,80.000000,0.333333,-0.395032,1.858480,-4.990911,62,0,0.000000,1.000000,0.875892,0.482508,-0.862334,-0.288987,0.957333,0.0,14.420785,-18.881958
2018-03-03 01:00:00,1,2018-03-03 1:00,-2.600000,3.658954,-4.218248e-01,54.666667,0.633333,-10.433333,38.333333,-2.216667,3.399868,-1.528627,58.000000,0.616667,-9.350000,45.000000,-1.333333,3.317941,-2.552299,58.333333,0.558333,-8.441667,55.166667,0.216667,3.459871,-3.321438,69.125000,0.554167,-5.008333,50.250000,-2.019478,79.435790,0.369300,-0.256762,1.698097,-5.103545,62,1,0.258819,0.965926,0.875892,0.482508,-0.865901,0.224661,0.974437,0.0,11.429188,-22.091333
2018-03-03 02:00:00,2,2018-03-03 2:00,-2.900000,3.333333,-5.180000e-16,53.666667,0.666667,-10.933333,36.333333,-2.450000,3.267536,-1.067963,56.166667,0.633333,-9.966667,42.000000,-1.608333,3.126905,-2.406940,58.000000,0.575000,-8.775000,52.916667,-0.025000,3.461898,-3.235733,67.333333,0.554167,-5.566667,51.291667,-1.976883,79.072872,0.358062,-0.048958,1.750787,-5.122597,62,2,0.500000,0.866025,0.875892,0.482508,-0.815907,0.628564,0.777757,0.0,9.769403,-23.598013
2018-03-03 03:00:00,3,2018-03-03 3:00,-3.166667,2.914400,2.964175e-01,51.666667,0.666667,-11.666667,37.333333,-2.733333,3.074668,-0.416237,54.333333,0.633333,-10.650000,39.666667,-1.933333,3.011320,-2.065008,57.500000,0.583333,-9.200000,50.750000,-0.254167,3.614586,-3.138035,65.375000,0.550000,-6.145833,52.208333,-2.000000,80.000000,0.333333,-0.099438,1.897396,-4.990911,62,3,0.707107,0.707107,0.875892,0.482508,-0.715760,0.844430,0.535666,0.0,11.219242,-25.449844
2018-03-03 04:00:00,4,2018-03-03 4:00,-3.633333,1.947701,6.539049e-01,52.333333,0.633333,-11.966667,42.000000,-3.116667,2.803328,0.116040,53.500000,0.633333,-11.200000,40.166667,-2.308333,2.915302,-1.505790,57.416667,0.591667,-9.575000,49.333333,-0.520833,3.739582,-3.093349,63.625000,0.550000,-6.712500,53.416667,-2.152114,82.698177,0.322086,-0.622530,2.032728,-4.699921,62,4,0.866025,0.500000,0.875892,0.482508,-0.572283,0.946030,0.324080,0.0,15.702102,-23.892936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 19:00:00,25555,2021-01-31 19:00,3.466667,1.104349,-3.534208e-01,63.000000,0.000000,-3.066667,131.333333,5.433333,1.248482,-1.861388,56.000000,0.033333,-2.850000,146.166667,4.600000,0.067148,-1.897564,63.416667,0.400000,-1.991667,110.666667,-0.504167,-0.153580,-0.984725,69.958333,0.420833,-5.525000,70.166667,5.513659,77.698089,0.990358,-3.038161,0.025389,1.924737,31,19,-0.965926,0.258819,0.508671,0.860961,-0.270730,-0.982466,-0.186443,0.0,98.210740,8.383643
2021-01-31 20:00:00,25556,2021-01-31 20:00,1.366667,0.443800,1.707800e-01,73.000000,0.000000,-3.033333,122.666667,4.133333,1.031869,-1.015664,60.500000,0.033333,-3.200000,139.166667,4.541667,0.324873,-1.749413,63.916667,0.316667,-1.966667,116.750000,-0.250000,-0.142179,-1.003402,70.583333,0.420833,-5.170833,73.916667,5.616223,79.424636,0.993445,-3.195202,0.471179,2.333071,31,20,-0.866025,0.500000,0.508671,0.860961,-0.463159,-0.999255,-0.038597,0.0,88.276383,15.745215
2021-01-31 21:00:00,25557,2021-01-31 21:00,-0.366667,0.224539,2.505847e-01,80.333333,0.000000,-3.333333,115.666667,2.633333,0.597777,-0

In [281]:
model=SARIMAX(train_dangjin_floating['energy'],exog=train_dangjin_floating[['principal component1','principal component2']],order=(5,1,0),
                       seasonal_order=(2, 0, 0, 12),enforce_invertibiblity=False)

results=model.fit()

results.summary()

/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/nayounlee2/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                             energy   No. Observations:                25536
Model:             SARIMAX(5, 1, 0)x(2, 0, 0, 12)   Log Likelihood             -138913.075
Date:                            Wed, 02 Jun 2021   AIC                         277846.150
Time:                                    14:41:47   BIC                         277927.628
Sample:                                         0   HQIC                        277872.495
                                          - 25536                                         
Covariance Type:                              opg                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
principal component1     0.1827      0.067      2.739      0.006       0.052       0.313
principal component2    -1.2496      0.103    -12.181      0.000      -1.451      -1.049
ar.L1                    0.3268      0.003     95.534      0.000       0.320       0.333
ar.L2                    0.0305      0.004      8.158      0.000       0.023       0.038
ar.L3                   -0.0284      0.004     -6.803      0.000      -0.037      -0.020
ar.L4                   -0.0631      0.005    -13.303      0.000      -0.072      -0.054
ar.L5                   -0.1242      0.005    -23.852      0.000      -0.134      -0.114
ar.S.L12                -0.0561      0.021     -2.694      0.007      -0.097      -0.015
ar.S.L24                 0.4231      0.003    123.773      0.000       0.416       0.430
sigma2                3109.3224     13.356    232.802      0.000    3083.145    3135.500
===================================================================================
Ljung-Box (L1) (Q):                   7.06   Jarque-Bera (JB):             79828.43
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               0.98   Skew:                            -0.08
Prob(H) (two-sided):                  0.32   Kurtosis:                        11.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [282]:
fcast4=results.predict(0,len(test_dangjin_floating),exog=test_dangjin_floating[['principal component1','principal component2']]).rename("FINAL SARIMAX FORECAST")

## Submission 파일로 바꾸기

In [283]:
submission = pd.read_csv('sample_submission.csv')

In [284]:
submission.iloc[:671]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0
...,...,...,...,...,...
666,2021-02-28 19:00:00,0,0,0,0
667,2021-02-28 20:00:00,0,0,0,0
668,2021-02-28 21:00:00,0,0,0,0
669,2021-02-28 22:00:00,0,0,0,0


In [285]:
fcast1=fcast1.reset_index()
fcast1=fcast1['FINAL SARIMAX FORECAST']

In [286]:
fcast2=fcast2.reset_index()
fcast2=fcast2['FINAL SARIMAX FORECAST']

In [287]:
fcast3=fcast3.reset_index()
fcast3=fcast3['FINAL SARIMAX FORECAST']

In [293]:
fcast4=fcast4.reset_index()
fcast4=fcast4['FINAL SARIMAX FORECAST']

In [294]:
submission['ulsan'] = fcast1

In [295]:
submission['dangjin'] = fcast2

In [296]:
submission['dangjin_warehouse'] = fcast3

In [297]:
submission['dangjin_floating'] = fcast4

In [298]:
submission.iloc[:671]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,26.215497,19.793938,27.737393,21.622630
1,2021-02-01 02:00:00,3.416561,2.461668,3.421702,-1.636494
2,2021-02-01 03:00:00,0.375783,0.251942,0.388746,-1.300520
3,2021-02-01 04:00:00,2.023717,1.524954,2.191048,-1.181914
4,2021-02-01 05:00:00,-1.808873,-1.327428,-1.930356,-0.913370
...,...,...,...,...,...
666,2021-02-28 19:00:00,87.446996,6.203040,34.139530,11.208255
667,2021-02-28 20:00:00,-19.511281,-0.015926,-18.654792,-1.403449
668,2021-02-28 21:00:00,-1.654321,-1.986333,-5.411013,5.157382
669,2021-02-28 22:00:00,14.345160,-2.826135,-3.532482,7.893012


In [299]:
submission.to_csv('submission_0602.csv', index=False)